## CO2sys with TEOS-10 support

In 2010, new definitions, Conservative Temperature (Tcsv) and Absolute Salinity (SA), became the new adopted standard in oceanography for temperature and salinity. 

CO2sys software, until now, accepted only in-situ temperature and practical salinity as input. CO2sys software has been extended to implement new oceanographic standards for ocean temperature and salinity (Thermodynamic Equation of Seawater, TEOS-10).

#### The aim of this notebook is to show how to use this extension through several examples.

## A two step procedure
Due to ascendant compatibility requirements, TEOS support has not been deeply integrated into CO2SYS. Instead, we chose to provide only a set of conversion routines in Matlab.

When using TEOS-10 temperature and salinity, a two step procedure is required from user's program:
1. call conversion routines for temperature and salinity input
2. call the main CO2sys routine (unchanged) with converted input

## Salinity conversion

For salinity conversion, there exist two different methods:

1. When chemical composition is not known
salinity conversion is based on temperature, salinity, sea water pressure and geographic location (given by latitude and longitude). Geographic location is used to infer sea water chemical composition based on ocean climatology.

2. When chemical composition is known
salinity conversion is based on temperature, salinity, sea water pressure, total alkalinity, dissolved inorganic carbon, and nitrate and silicate concentrations

The first example will illustrate the case when chemical concentrations are not known but geographic location is. The second example will illustrate the second case.


### Input parameters
The following values will be used as input throughout the exemples in this notebook.

In [1]:
% Input parameters
TAlk = 2300;        % µmol/kg
DIC = 2000;
SA = 35;            % Absolute Salinity
CsvTin = 18;        % Convervative temperature at input conditions
Pin = 0;            % Pressure at input conditions

% Nutrient concentration
TSil = 60;           % Total Silicate (µmol/kg)
TPhos = 2;           % Total Phosphate (µmol/kg)

% prevent specific warning message
warning ("off", "Matlab-style short-circuit operation performed for operator &")


## 1st example

### 1st step: conversion from TEOS-10 to EOS-80
Salinity conversion depends on depth (pressure) and geographic location: If location is not given, an arbitrary location is chosen: the mid equatorial atlantic ocean. <br> Note that this implies an error on computed SA ranging from 0 up to 0.02 g/kg

Temperature conversion depends on pressure only

In [2]:
% the following function call converts
%    - Temperature :  from Conservative to in-situ 
%    - Salinity    :  from Absolute (g/kg) to Practical (psu)

# Example when longitude and latitude are known
long = -10;    # degrees East
lat = 45;      # degrees North
[Tin, SP] = teos2eos_geo(SA, CsvTin, Pin, long, lat);
"geographic location : North Atlantic"
Tin
SP

# Example when longitude and latitude are NOT known
# "geographic location : unknown"
# [Tin, SP] = teos2eos_geo(SA, CTin, Pin, [], []);


ans = geographic location : North Atlantic
Tin =  18.004
SP =  34.836


### 2nd step: CO2sys computation with converted S and T
In this step, we just call CO2SYS() routine as usually.

In [3]:
[DATA,HEADERS] = CO2SYS(TAlk,DIC,1,2,SP,Tin,0,Pin,0,TSil,TPhos,1,10,1);
HEADERS(1:8)
DATA(1:8)

ans = 
{
  [1,1] = TAlk
  [2,1] = TCO2
  [3,1] = pHin
  [4,1] = pCO2in
  [5,1] = fCO2in
  [6,1] = HCO3in
  [7,1] = CO3in
  [8,1] = CO2in
}
ans =

 Columns 1 through 6:

   2300.0000   2000.0000      8.1470    302.5164    301.4646   1782.1221

 Columns 7 and 8:

    207.5333     10.3446



## 2nd example

### 1st step: Alternative conversion from TEOS-10 to EOS-80
This conversion depends on chemical composition of seawater: alcalinity, carbon, nitrate and silicate. Note that the nitrate concentration may be inferred from the Phosphate concentration, using Redfield ratio.

In [6]:
% Nitrate: one may use Redfield ratio and Phosphate concentration
NH3 = TPhos * 16;
[Tin, SP] = teos2eos_chem(SA, CsvTin, Pin, TAlk, DIC, NH3, TSil);
Tin
SP

Tin =  18.004
SP =  34.831


### 2nd step: CO2sys computation with converted S and T
In this step, we just call CO2SYS() routine as usually.

In [5]:
[DATA,HEADERS] = CO2SYS(TAlk,DIC,1,2,SP,Tin,0,Pin,0,TSil,TPhos,1,10,1);
HEADERS(1:8)
DATA(1:8)

ans = 
{
  [1,1] = TAlk
  [2,1] = TCO2
  [3,1] = pHin
  [4,1] = pCO2in
  [5,1] = fCO2in
  [6,1] = HCO3in
  [7,1] = CO3in
  [8,1] = CO2in
}
ans =

 Columns 1 through 6:

   2300.0000   2000.0000      8.1471    302.4809    301.4292   1782.1181

 Columns 7 and 8:

    207.5383     10.3436

